
# ✈️ Flight Delay Prediction Using PySpark and Google Cloud

This notebook demonstrates an end-to-end big data pipeline for predicting flight delays using PySpark on Google Cloud Platform (GCP). The dataset used contains millions of airline records from the Bureau of Transportation Statistics.


In [ ]:

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col, when


In [ ]:

spark = SparkSession.builder.appName("FlightDelayPrediction").getOrCreate()


In [ ]:

# Replace with your GCS path
data = spark.read.csv("gs://your-bucket/airline_data.csv", header=True, inferSchema=True)
data.printSchema()
data.show(5)


In [ ]:

# Create binary target variable
data = data.withColumn("IsDelayed", when(col("ArrDelay") > 15, 1).otherwise(0))

# Select relevant columns
columns = ["Carrier", "DepTime", "Distance", "DayOfWeek"]
data = data.select(columns + ["IsDelayed"]).dropna()


In [ ]:

# Convert DepTime to hour
data = data.withColumn("DepHour", (col("DepTime") / 100).cast("int")).drop("DepTime")

# StringIndexer for categorical features
carrier_indexer = StringIndexer(inputCol="Carrier", outputCol="CarrierIndexed")

# Assemble features
assembler = VectorAssembler(
    inputCols=["CarrierIndexed", "DepHour", "Distance", "DayOfWeek"],
    outputCol="features_raw"
)

# Scale features
scaler = StandardScaler(inputCol="features_raw", outputCol="features")

# Logistic Regression Model
lr = LogisticRegression(labelCol="IsDelayed", featuresCol="features")

# Pipeline
pipeline = Pipeline(stages=[carrier_indexer, assembler, scaler, lr])


In [ ]:

train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(train_data)
predictions = model.transform(test_data)


In [ ]:

evaluator = BinaryClassificationEvaluator(labelCol="IsDelayed")
accuracy = evaluator.evaluate(predictions)
print(f"Test ROC-AUC: {accuracy:.4f}")



## ✅ Conclusion

- Successfully built a distributed logistic regression pipeline using PySpark and GCP
- Achieved solid ROC-AUC score using basic airline features
- Project demonstrates practical use of cloud-scale data engineering and ML

Further improvements could include trying other models (Random Forest, GBT) or tuning hyperparameters.
